In [1]:
import sys
sys.path.append('../src')

In [2]:
from covid import daily_snapshot
from pathlib import Path

import population as pop
import pandas as pd
import requests

In [3]:
pd.set_option('max_rows', 100) 

In [4]:
date = input('date in mm-dd-yyyy format')

date in mm-dd-yyyy format 04-03-2020


In [5]:
state = 'New Jersey'

### Quick glance at the data

In [6]:
df = daily_snapshot(date)
df.head()

,Unnamed: 0,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
FIPS,,,,,,,,,,,,
45001.0,NaN,Abbeville,South Carolina,US,2020-04-03 22:46:37,34.223334,-82.461707,6,0,0,0,"Abbeville, South Carolina, US"
22001.0,NaN,Acadia,Louisiana,US,2020-04-03 22:46:37,30.295065,-92.414197,72,1,0,0,"Acadia, Louisiana, US"
51001.0,NaN,Accomack,Virginia,US,2020-04-03 22:46:37,37.767072,-75.632346,8,0,0,0,"Accomack, Virginia, US"
16001.0,NaN,Ada,Idaho,US,2020-04-03 22:46:37,43.452658,-116.241552,307,3,0,0,"Ada, Idaho, US"
19001.0,NaN,Adair,Iowa,US,2020-04-03 22:46:37,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US"


In [7]:
df[df.Country_Region=='US'].describe()

,Unnamed: 0,Lat,Long_,Confirmed,Deaths,Recovered,Active
count,0.0,2322.000000,2322.000000,2368.000000,2368.000000,2368.000000,2368.0
mean,NaN,37.952568,-90.418414,116.379223,2.992821,4.099240,0.0
std,NaN,5.119809,14.284283,1292.875036,39.369604,199.477621,0.0
min,NaN,13.444300,-159.596679,0.000000,0.000000,0.000000,0.0
25%,NaN,34.245714,-95.931390,2.000000,0.000000,0.000000,0.0
50%,NaN,38.020890,-88.213113,7.000000,0.000000,0.000000,0.0
75%,NaN,41.496883,-82.338627,27.000000,1.000000,0.000000,0.0
max,NaN,65.508155,145.673900,57159.000000,1584.000000,9707.000000,0.0


### Aggregate by state

In [8]:
state_totals = df[df.Country_Region=='US'].groupby(by='Province_State').sum()

In [9]:
state_totals.describe()

,Unnamed: 0,Lat,Long_,Confirmed,Deaths,Recovered,Active
count,58.0,58.000000,58.000000,58.000000,58.000000,58.000000,58.0
mean,0.0,1519.411428,-3619.854453,4751.482759,122.189655,167.362069,0.0
std,0.0,1245.038349,3112.331438,14017.685794,395.557056,1274.591544,0.0
min,0.0,0.000000,-14651.797929,0.000000,0.000000,0.000000,0.0
25%,0.0,511.030915,-5627.524695,336.500000,6.250000,0.000000,0.0
50%,0.0,1287.901244,-3170.306929,944.500000,23.500000,0.000000,0.0
75%,0.0,2515.280771,-958.649806,3405.750000,95.500000,0.000000,0.0
max,0.0,4798.990598,145.673900,102987.000000,2935.000000,9707.000000,0.0


In [10]:
state = 'New Jersey'
state_covid = df[df.Province_State == state]
state_columns=['Admin2', 'Confirmed', 'Deaths', 'Recovered', 'Active', ]

In [11]:
state_covid[state_columns].sort_values(by='Confirmed', ascending=False)

,Admin2,Confirmed,Deaths,Recovered,Active
FIPS,,,,,
34003.0,Bergen,4866,132,0,0
90034.0,Unassigned,4808,4,0,0
34013.0,Essex,3067,118,0,0
34017.0,Hudson,2835,59,0,0
34039.0,Union,2487,45,0,0
34031.0,Passaic,2216,34,0,0
34023.0,Middlesex,2125,56,0,0
34025.0,Monmouth,1743,48,0,0
34029.0,Ocean,1685,45,0,0


In [12]:
state_covid[state_columns].sort_values(by='Deaths', ascending=False)

,Admin2,Confirmed,Deaths,Recovered,Active
FIPS,,,,,
34003.0,Bergen,4866,132,0,0
34013.0,Essex,3067,118,0,0
34017.0,Hudson,2835,59,0,0
34023.0,Middlesex,2125,56,0,0
34025.0,Monmouth,1743,48,0,0
34039.0,Union,2487,45,0,0
34029.0,Ocean,1685,45,0,0
34027.0,Morris,1298,43,0,0
34031.0,Passaic,2216,34,0,0


In [13]:
nj_county_population_url = 'https://www.newjersey-demographics.com/counties_by_population'

In [14]:
nj_pop = pop.nj_county_population(nj_county_population_url)

In [15]:
nj_pop

,county,pop2019
Rank,,
1,Bergen,929999
2,Middlesex,826698
3,Essex,793555
4,Hudson,668631
5,Monmouth,623387
6,Ocean,591939
7,Union,553066
8,Camden,507367
9,Passaic,504041


In [16]:
state_with_population = pd.merge(state_covid[state_columns], nj_pop,
                                 how='outer', left_on='Admin2', right_on='county')

state_with_population['fraction_confirmed'] = state_with_population['Confirmed'] / state_with_population['pop2019']   * 1000.0
state_with_population['deaths']             = state_with_population['Deaths']    / state_with_population['pop2019']   * 1000.0
state_with_population['death_rate?']        = state_with_population['Deaths']    / state_with_population['Confirmed'] * 1000.0

In [17]:
sort_by = input("column to sort Confirmed, Deaths, county, pop2019, fraction_confirmed, deaths, death_rate?")

column to sort Confirmed, Deaths, county, pop2019, fraction_confirmed, deaths, death_rate? fraction_confirmed


In [18]:
state_with_population.sort_values(by=sort_by, ascending=False)

,Admin2,Confirmed,Deaths,Recovered,Active,county,pop2019,fraction_confirmed,deaths,death_rate?
1,Bergen,4866,132,0,0,Bergen,929999.0,5.232264,0.141936,27.127004
20,Union,2487,45,0,0,Union,553066.0,4.496751,0.081365,18.094089
15,Passaic,2216,34,0,0,Passaic,504041.0,4.396468,0.067455,15.342960
8,Hudson,2835,59,0,0,Hudson,668631.0,4.240007,0.088240,20.811287
6,Essex,3067,118,0,0,Essex,793555.0,3.864886,0.148698,38.474079
14,Ocean,1685,45,0,0,Ocean,591939.0,2.846577,0.076021,26.706231
12,Monmouth,1743,48,0,0,Monmouth,623387.0,2.796016,0.076999,27.538726
13,Morris,1298,43,0,0,Morris,494383.0,2.625495,0.086977,33.127889
11,Middlesex,2125,56,0,0,Middlesex,826698.0,2.570467,0.067739,26.352941
17,Somerset,641,23,0,0,Somerset,330176.0,1.941389,0.069660,35.881435
